<a href="https://colab.research.google.com/github/Isafon/ECE528/blob/main/ECE528_ASN3_Q2a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECE528 Lab 3 Q2a - Isa Fontana

#### Q2a: Adding in TensorFlow Lite Model

## Imports!

In [1]:
import os, io, zipfile, numpy as np, pandas as pd, tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import matplotlib.pyplot as plt

np.random.seed(42)
tf.random.set_seed(42)
print("TF version:", tf.__version__)

TF version: 2.19.0


## Choose File

In [2]:
from google.colab import files
uploaded = files.upload()

Saving archive.zip to archive.zip


## Unzip the File

In [3]:
# Create a working folder
DATA_DIR = "./data_asl"
os.makedirs(DATA_DIR, exist_ok=True)

# If a zip was uploaded, extract it
for fname in uploaded.keys():
    if fname.lower().endswith(".zip"):
        with zipfile.ZipFile(io.BytesIO(uploaded[fname]), 'r') as zf:
            zf.extractall(DATA_DIR)
        print(f"Extracted: {fname} -> {DATA_DIR}")

# Figure out where the CSVs ended up (root or inside DATA_DIR)
candidates = [
    "sign_mnist_train.csv",
    "sign_mnist_test.csv",
    os.path.join(DATA_DIR, "sign_mnist_train.csv"),
    os.path.join(DATA_DIR, "sign_mnist_test.csv"),
]

# Build resolved paths
train_csv, test_csv = None, None
for c in candidates:
    if c.endswith("sign_mnist_train.csv") and os.path.exists(c):
        train_csv = c
    if c.endswith("sign_mnist_test.csv") and os.path.exists(c):
        test_csv = c

assert train_csv and test_csv, "Could not find the CSVs. Re-upload the zip or both CSV files."

print("Train CSV:", train_csv)
print("Test  CSV:", test_csv)

Extracted: archive.zip -> ./data_asl
Train CSV: ./data_asl/sign_mnist_train.csv
Test  CSV: ./data_asl/sign_mnist_test.csv


## Load Data

In [4]:
# Load CSVs
train_df = pd.read_csv(train_csv)
test_df  = pd.read_csv(test_csv)

# Separate labels and pixels
y_train_raw = train_df.pop('label').values
y_test_raw  = test_df.pop('label').values

x_train = train_df.values.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test  = test_df.values.reshape(-1, 28, 28, 1).astype("float32") / 255.0

# Make labels contiguous (handles “missing J and Z”)
uniq = np.unique(np.concatenate([y_train_raw, y_test_raw]))
remap = {old:i for i, old in enumerate(sorted(uniq))}
y_train = np.array([remap[v] for v in y_train_raw])
y_test  = np.array([remap[v] for v in y_test_raw])

num_classes = len(uniq)  # should be 24
print("Shapes:", x_train.shape, x_test.shape)
print("Classes detected:", num_classes, "Original label ids:", uniq)

Shapes: (27455, 28, 28, 1) (7172, 28, 28, 1)
Classes detected: 24 Original label ids: [ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]


## Model It

In [5]:
def CBR(filters):
    # Conv -> BatchNorm -> ReLU (BN immediately before activation per instructions)
    return tf.keras.Sequential([
        layers.Conv2D(filters, 3, padding="same", use_bias=False),
        layers.BatchNormalization(),
        layers.ReLU(),
    ])

inputs = layers.Input((28, 28, 1))
x = CBR(32)(inputs);  x = CBR(32)(x);  x = layers.MaxPool2D()(x);  x = layers.Dropout(0.25)(x)
x = CBR(64)(x);       x = CBR(64)(x);  x = layers.MaxPool2D()(x);  x = layers.Dropout(0.25)(x)
x = CBR(128)(x);      x = layers.Conv2D(128, 3, padding="same", use_bias=False)(x)
x = layers.BatchNormalization()(x); x = layers.ReLU()(x)
x = layers.GlobalAveragePooling2D()(x); x = layers.Dropout(0.40)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = models.Model(inputs, outputs, name="asl_mnist_cnn")
model.summary()

Model: "asl_mnist_cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 28, 28, 32)     │           416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 28, 28, 32)     │         9,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_2 (Sequential)       │ (None, 14, 14, 64)     │        18,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_3 (Sequential)       │ (None, 14, 14, 64)     │        37,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_4 (Sequential)       │ (None, 7, 7, 128)      │        74,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 7, 7, 128)      │       147,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_5 (ReLU)                  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 24)             │         3,096 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 290,872 (1.11 MB)

 Trainable params: 289,976 (1.11 MB)

 Non-trainable params: 896 (3.50 KB)

## Train It

In [6]:
from tensorflow.keras import callbacks

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

rlrop = callbacks.ReduceLROnPlateau(
    monitor="val_accuracy", factor=0.5, patience=2, min_lr=1e-5, verbose=1
)
es = callbacks.EarlyStopping(
    monitor="val_accuracy", patience=6, restore_best_weights=True, verbose=1
)

history = model.fit(
    x_train, y_train,
    epochs=60,           # longer run; ES will stop early
    batch_size=128,
    validation_split=0.10,     # from TRAIN only
    callbacks=[rlrop, es],
    verbose=2
)

Epoch 1/60
194/194 - 21s - 111ms/step - accuracy: 0.6083 - loss: 1.3851 - val_accuracy: 0.0572 - val_loss: 4.1373 - learning_rate: 1.0000e-03
Epoch 2/60
194/194 - 1s - 6ms/step - accuracy: 0.9524 - loss: 0.2442 - val_accuracy: 0.2753 - val_loss: 2.3629 - learning_rate: 1.0000e-03
Epoch 3/60
194/194 - 1s - 5ms/step - accuracy: 0.9904 - loss: 0.0735 - val_accuracy: 0.9811 - val_loss: 0.1048 - learning_rate: 1.0000e-03
Epoch 4/60
194/194 - 1s - 5ms/step - accuracy: 0.9971 - loss: 0.0353 - val_accuracy: 0.9996 - val_loss: 0.0163 - learning_rate: 1.0000e-03
Epoch 5/60
194/194 - 1s - 5ms/step - accuracy: 0.9980 - loss: 0.0203 - val_accuracy: 0.9996 - val_loss: 0.0062 - learning_rate: 1.0000e-03
Epoch 6/60

Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
194/194 - 1s - 5ms/step - accuracy: 0.9985 - loss: 0.0155 - val_accuracy: 0.9938 - val_loss: 0.0183 - learning_rate: 1.0000e-03
Epoch 7/60
194/194 - 1s - 5ms/step - accuracy: 0.9991 - loss: 0.0092 - val_accuracy: 1

In [7]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print("Q1 Test accuracy:", round(float(test_acc), 4))

Q1 Test accuracy: 0.9974


## Accuracy Overall (Proof)

In [8]:
# === Q1: summarize training/validation accuracy over all completed epochs ===
import numpy as np

hist = history.history
# Works with both old/new key names
train_key = 'accuracy' if 'accuracy' in hist else 'acc'
val_key   = 'val_accuracy' if 'val_accuracy' in hist else 'val_acc'

train_acc = np.array(hist[train_key], dtype=float)
val_acc   = np.array(hist[val_key],   dtype=float)

epochs_run = len(val_acc)
best_idx   = int(np.argmax(val_acc))           # 0-based
best_epoch = best_idx + 1                      # 1-based
best_val   = float(val_acc[best_idx])
mean_val   = float(val_acc.mean())
mean_train = float(train_acc.mean())

print(f"Epochs completed: {epochs_run}")
print(f"Mean TRAIN accuracy over epochs: {mean_train:.4f}")
print(f"Mean VAL   accuracy over epochs: {mean_val:.4f}")
print(f"Best VAL accuracy: {best_val:.4f} (epoch {best_epoch})")
print(f"Last VAL accuracy: {val_acc[-1]:.4f}")

try:
    print(f"TEST accuracy: {float(test_acc):.4f}")
except NameError:
    print("TEST accuracy: (run your evaluate cell to show this)")

# Quick pass/fail for the assignment target
meets_target = (mean_val >= 0.92) or ('test_acc' in globals() and float(test_acc) >= 0.92)
print("Meets ≥92% target (mean VAL or TEST):", "Yes" if meets_target else "Not yet")

Epochs completed: 13
Mean TRAIN accuracy over epochs: 0.9648
Mean VAL   accuracy over epochs: 0.8697
Best VAL accuracy: 1.0000 (epoch 7)
Last VAL accuracy: 1.0000
TEST accuracy: 0.9974
Meets ≥92% target (mean VAL or TEST): Yes


## Save the Model

In [9]:
model.save("asl_mnist_baseline.keras")
print("Saved: asl_mnist_baseline.keras")

Saved: asl_mnist_baseline.keras


## Q2a New Stuff Here

In [10]:
SAVE_PATH = "asl_mnist_baseline.keras"
if "model" in globals():
    model.save(SAVE_PATH)
    print(f"Saved trained model to {SAVE_PATH}")
else:
    print("No in-memory model found; loading from disk...")
    model = tf.keras.models.load_model(SAVE_PATH)
    print(f"Loaded model from {SAVE_PATH}")

# Sanity: have test tensors?
assert "x_test" in globals() and "y_test" in globals(), "x_test / y_test not found (rerun Q1 data cells)."
print("Test set:", x_test.shape, y_test.shape)

Saved trained model to asl_mnist_baseline.keras
Test set: (7172, 28, 28, 1) (7172,)


## Helper Method

In [23]:
def tflite_accuracy(tflite_path, x, y, batch_size=128):
    import numpy as np
    import tensorflow as tf
    from tensorflow.lite.python.interpreter import Interpreter  # deprecation warning is fine for now

    x = x.astype(np.float32)
    interp = Interpreter(model_path=tflite_path)
    # We’ll (re)allocate per batch, so no allocate_tensors() yet

    # Read expected spatial dims
    in_det  = interp.get_input_details()[0]
    out_det = interp.get_output_details()[0]
    # Safe default if the model stores a dummy batch dim
    H, W, C = int(in_det['shape'][1]), int(in_det['shape'][2]), int(in_det['shape'][3])

    n = len(x)
    correct = 0
    start = 0
    while start < n:
        end = min(start + batch_size, n)
        batch = x[start:end]

        # ---- KEY: resize to current batch size, then allocate, then run
        interp.resize_tensor_input(in_det['index'], [end - start, H, W, C], strict=False)
        interp.allocate_tensors()
        interp.set_tensor(in_det['index'], batch)
        interp.invoke()
        preds = interp.get_tensor(out_det['index'])   # shape [B, 24]
        correct += int((np.argmax(preds, axis=1) == y[start:end]).sum())
        start = end

    return correct / n

## Convert to float TFLite

In [24]:
# Float TFLite (baseline)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_float = converter.convert()
open("s_mnist.tflite", "wb").write(tflite_float)

print("Wrote s_mnist.tflite")

Saved artifact at '/tmp/tmp64afuzdp'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 24), dtype=tf.float32, name=None)
Captures:
  139118138538128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139118138539856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139118131331536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139118138536976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139118138539472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139118131331152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139118131334032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139118131333840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139118131332496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139118131332880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139118131335184

## Convert with Dynamic Range

In [25]:
# Dynamic range quantization (weights → int8; activations quantized dynamically at runtime)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_dyn = converter.convert()
open("s_mnist_quant_dyn.tflite", "wb").write(tflite_dyn)

print("Wrote s_mnist_quant_dyn.tflite")

Saved artifact at '/tmp/tmpstud0atn'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 24), dtype=tf.float32, name=None)
Captures:
  139118138538128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139118138539856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139118131331536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139118138536976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139118138539472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139118131331152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139118131334032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139118131333840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139118131332496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139118131332880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139118131335184

## Evaluate BOTH Models

In [26]:
acc_float = tflite_accuracy("s_mnist.tflite", x_test, y_test)       # or tflite_accuracy_b1(...)
acc_dyn   = tflite_accuracy("s_mnist_quant_dyn.tflite", x_test, y_test)

import os
mb = lambda p: os.path.getsize(p)/1024/1024
print(f"Float TFLite : acc={acc_float:.4f} | size={mb('s_mnist.tflite'):.2f} MB")
print(f"Dynamic-Range: acc={acc_dyn:.4f} | size={mb('s_mnist_quant_dyn.tflite'):.2f} MB")
print(f"Δacc (dyn - float): {acc_dyn-acc_float:+.4f}")
print(f"Size reduction     : {(1 - mb('s_mnist_quant_dyn.tflite')/mb('s_mnist.tflite'))*100:.1f}%")

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


Float TFLite : acc=0.9974 | size=1.11 MB
Dynamic-Range: acc=0.9974 | size=0.29 MB
Δacc (dyn - float): +0.0000
Size reduction     : 74.0%


## Isa's Write Up

#### I converted my ASL MNIST CNN to TensorFlow Lite and evaluated both the float TFLite model and a dynamic-range quantized version. The float model (s_mnist.tflite) achieved 99.74% test accuracy and is around 1.11 MB. The dynamic-range model (s_mnist_quant_dyn.tflite) also achieved 99.74% accuracy, while shrinking to around 0.29 MB (~74% smaller). Dynamic range quantization statically quantizes weights to int8 and dynamically quantizes activations at runtime, which typically improves CPU latency with little/no accuracy loss. In my case there was no measurable accuracy drop and a large size reduction, so this is a good trade-off for embedded deployment.